Chia tập train, dev, test data

In [ ]:
# Split the subset by rating to create new train, val, and test splits
import collections
import numpy as np
import pandas as pd

by_rating = collections.defaultdict(list)
for _, row in review_subset.iterrows():
    by_rating[row.rating].append(row.to_dict())
    
# Create split data
final_list = []
np.random.seed(args.seed)

for _, item_list in sorted(by_rating.items()):
    np.random.shuffle(item_list)
    n_total = len(item_list)
    n_train = int(args.train_proportion * n_total)
    n_val = int(args.val_proportion * n_total)
    n_test = int(args.test_proportion * n_total)

# Give data point a split attribute
for item in item_list[:n_train]:
    item['split'] = 'train'

for item in item_list[n_train:n_train+n_val]:
    item['split'] = 'val'

for item in item_list[n_train+n_val:n_train+n_val+n_test]:
    item['split'] = 'test'

# Add to final list
final_list.extend(item_list)
final_reviews = pd.DataFrame(final_list)

Tiền xử lý dữ liệu


In [ ]:
import re

def preprocess(text):
    text = re.sub(r"([.,!?])", r" \1 ", text)
    text = re.sub(r"[^a­zA­Z.,!?]+", r" ", text)
    return text

Lớp xử lý dataset

In [ ]:
from torch.utils.data import Dataset
class ReviewDataset(Dataset):
    def __init__(self, review_df, vectorizer):
        """
        Args:
        review_df (pandas.DataFrame): bộ dữ liệu
        vectorizer (ReviewVectorizer): vecto được tạo từ bộ dữ liệu
        """
        self.review_df = review_df
        self._vectorizer = vectorizer
        self.train_df = self.review_df[self.review_df.split=='train']
        self.train_size = len(self.train_df)
        self.val_df = self.review_df[self.review_df.split=='val']
        self.validation_size = len(self.val_df)
        self.test_df = self.review_df[self.review_df.split=='test']
        self.test_size = len(self.test_df)
        self._lookup_dict = {'train': (self.train_df, self.train_size),
                            'val': (self.val_df, self.validation_size),
                            'test': (self.test_df, self.test_size)}
        self.set_split('train')
    @classmethod
    def load_dataset_and_make_vectorizer(cls, review_csv):
        """Load data và tạo vecto
        Args:
        review_csv (str): tập dữ liệu
        Returns:
        một biến với dạng ReviewVectorizer
        """
        review_df = pd.read_csv(review_csv)
        return cls(review_df, ReviewVectorizer.from_dataframe(review_df))

    def get_vectorizer(self):
        """ returns một vecto """
        return self._vectorizer

    def set_split(self, split="train"):
        """ chọn dữ liệu được chia theo các cột
        Args:
        split (str): one of "train", "val", or "test"
        """
        self._target_split = split
        self._target_df, self._target_size = self._lookup_dict[split]

    def __len__(self):
        return self._target_size

    def __getitem__(self, index):
        """hàm chính trong lớp DataReview
        Args:
        index (int): index của data point
        Returns:
        một dictionary của một điểm dữ liệu
        """
        row = self._target_df.iloc[index]
        review_vector = \
        self._vectorizer.vectorize(row.review)
        rating_index = \
        self._vectorizer.rating_vocab.lookup_token(row.rating)
        return {'x_data': review_vector,
                'y_target': rating_index}

    def get_num_batches(self, batch_size):
        """Cung cấp batch size, cung cấp số lượng batch trong tập data
        Args:
        batch_size (int)
        Returns:
        số lượng batch trong data
        """
        return len(self) // batch_size

Lớp Vocabulary

In [ ]:
class Vocabulary(object):
    """Lớp xử lý văn bản và tạo vocabulary từ việc thực hiện việc mapping các từ sang một số thực đại diện cho nó"""
    def __init__(self, token_to_idx=None, add_unk=True, unk_token="<UNK>"):
        """
        Args:
        token_to_idx (dict): một dictionary mapping sẵn giữa từ và số thực, key : chỉ số index, value : từ 
        add_unk (bool): xác định xem có thêm ký tự unk vào hay không
        unk_token (str): ký tự unk 
        """
        if token_to_idx is None:
            token_to_idx = {}
        self._token_to_idx = token_to_idx
        self._idx_to_token = {idx: token
                              for token, idx in self._token_to_idx.items()}
        self._add_unk = add_unk
        self._unk_token = unk_token
        self.unk_index = 1
        if add_unk:
            self.unk_index = self.add_token(unk_token)

    def to_serializable(self):
        """ một từ điển có thể xử lý tuần tự """
        return {'token_to_idx': self._token_to_idx,
                'add_unk': self._add_unk,
                'unk_token': self._unk_token}

    @classmethod
    def from_serializable(cls, contents):
        """ khởi tạo vocabulary từ dictionary """
        return cls(**contents)
        
    def add_token(self, token):
        """mapping với từ mới
        Args:
        token (str): từ cần thêm vào vocabulary
        Returns:
        index (int): index của từ tương ứng
        """
        if token in self._token_to_idx:
            index = self._token_to_idx[token]
        else:
            index = len(self._token_to_idx)
            self._token_to_idx[token] = index
            self._idx_to_token[index] = token
        return index

    def lookup_token(self, token):
        """Truy vấn index của một từ
        Args:
        token (str): từ cần truy vấn 
        Returns:
        index (int): index ứng với từ cần truy vấn
        Notes:
        `unk_index` cần >=0 (unk đã được add vào trong vocabulary)
        cho trường hợp từ đó không xuất hiện
        """
        if self.add_unk:
            return self._token_to_idx.get(token, self.unk_index)
        else:
            return self._token_to_idx[token]

    def lookup_index(self, index):
        """truy vấn từ dựa trên index
        Args:
        index (int): index của từ cần truy vấn
        Returns:
        token (str): từ cần truy vấn
        Raises:
        KeyError: nếu index không nằm trong từ điển
        """
        if index not in self._idx_to_token:
            raise KeyError("the index (%d) is not in the Vocabulary" % index)
        return self._idx_to_token[index]
    def __str__(self):
        return "<Vocabulary(size=%d)>" % len(self)
    def __len__(self):
        return len(self._token_to_idx)

Lớp Vectorize

In [ ]:
from collections import Counter
class ReviewVectorizer(object):
    """ Vectorize sử dụng Vocabulary để tạo ra vecto"""
    def __init__(self, review_vocab, rating_vocab):
        """
        Args:
        review_vocab (Vocabulary): map từ sang số nguyên
        rating_vocab (Vocabulary): maps nhãn của lớp sang số nguyên
        """
        self.review_vocab = review_vocab
        self.rating_vocab = rating_vocab
    def vectorize(self, review):
        """Tạo vecto onehot
        Args:
        review (str): văn bản
        Returns:
        one_hot (np.ndarray): vecto onehot
        """
        one_hot = np.zeros(len(self.review_vocab), dtype=np.float32)
        for token in review.split(" "):
            if token not in string.punctuation:
                one_hot[self.review_vocab.lookup_token(token)] = 1
        return one_hot
    @classmethod
    def from_dataframe(cls, review_df, cutoff=25):
        """Tạo vecto từ dataframe
        Args:
        review_df (pandas.DataFrame): tập dữ liệu
        cutoff (int): tham số cutoff
        Returns:
        Review_Vocabulary
        """
        review_vocab = Vocabulary(add_unk=True)
        rating_vocab = Vocabulary(add_unk=False)

        # Add ratings
        for rating in sorted(set(review_df.rating)):
            rating_vocab.add_token(rating)
        # Add top words if count > provided count
        word_counts = Counter()
        for review in review_df.review:
            for word in review.split(" "):
                if word not in string.punctuation:
                    word_counts[word] += 1
        for word, count in word_counts.items():
            if count > cutoff:
                review_vocab.add_token(word)
        return cls(review_vocab, rating_vocab)
    @classmethod
    def from_serializable(cls, contents):
        """Tuần tự hóa
        Args:
        contents (dict): dictionary đã được tuần tự hóa
        Returns:
        Review_Vocabulary
        """
        review_vocab = Vocabulary.from_serializable(contents['review_vocab'])
        rating_vocab = Vocabulary.from_serializable(contents['rating_vocab'])
        return cls(review_vocab=review_vocab, rating_vocab=rating_vocab)
    def to_serializable(self):
        """Tạo dictionary để xử lý tuần tự
        Returns:
        contents (dict): dictionary có thể tuần tự hóa
        """
        return {'review_vocab': self.review_vocab.to_serializable(),
        'rating_vocab': self.rating_vocab.to_serializable()}

Lớp Dataloader

In [ ]:
def generate_batches(dataset, batch_size, shuffle=True,
    drop_last=True, device="cpu"):
    """
    đảm bảo mỗi vecto nằm ở trên cpu
    """
    dataloader = DataLoader(dataset=dataset, batch_size=batch_size,
    shuffle=shuffle, drop_last=drop_last)
    for data_dict in dataloader:
        out_data_dict = {}
        for name, tensor in data_dict.items():
            out_data_dict[name] = data_dict[name].to(device)
        yield out_data_dict


Xây dựng mạng nơ ron với cấu hình perceptron

In [ ]:
import torch.nn as nn
class ReviewClassifier(nn.Module):
    def __init__(self, num_feature) :
        super(ReviewClassifier, self).__init__()

        self.linear = nn.Linear(in_features=num_feature, out_features=1)
    
    def forward(self,input,apply_sigmoid = False):
        output = self.linear(input).squeeze()
        if apply_sigmoid:
            output = nn.Sigmoid(output)
        return output


Các hyperparameter

In [ ]:
from argparse import Namespace
args = Namespace(
    # Data and path information
    frequency_cutoff=25,
    model_state_file='model.pth',
    review_csv='data/yelp/reviews_with_splits_lite.csv',
    save_dir='model_storage/ch3/yelp/',
    vectorizer_file='vectorizer.json',
    # No model hyperparameters
    # Training hyperparameters
    batch_size=128,
    early_stopping_criteria=5,
    learning_rate=0.001,
    num_epochs=100,
    seed=1337,
    # Runtime options omitted for space
)